In [19]:
import pandas as pd

df_scrap  = pd.read_csv('./data/Propuesta_hoteles_ptvaras2024.csv', index_col=0)
df_procesado = pd.read_csv('./data/hoteles_puerto_varas_prototipo.csv', index_col=0)
# Hacer dummy las categorias pero sin perder esa columna
categorias = df_procesado['categorias']
df_procesado = pd.get_dummies(df_procesado, columns=['categorias'])
# Agregar las categorias a df_procesado
df_procesado['categorias'] = categorias



,nombre,localidad,score,reviews,latitud,longitud,categorias_Excellent,categorias_Very good,categorias
0,Silvestre Cabanas,Los Colonos 1183 Puerto Varas Chile,4.5,3,-41.329796,-72.961064,True,False,Excellent
1,Puerto Chico Hotel,Avenida Los Colonos no. 60 Puerto Varas Chile,4.0,263,-41.328733,-72.960613,False,True,Very good
2,Park Inn By Radisson Puerto Varas,La Paz 471 Puerto Varas Chile,3.5,531,-41.314910,-72.985654,False,True,Very good
3,Hotel Puelche,Imperial 695 Puerto Varas Chile,4.5,347,-41.322458,-72.976830,True,False,Excellent
4,Radisson Hotel Puerto Varas,Del Salvador 024 Puerto Varas Chile,4.5,861,-41.317983,-72.981778,True,False,Excellent
5,Hotel Awa,Ruta 225 Km. 27 Sector Los Riscos Puerto Var...,4.5,339,-41.317802,-72.982907,True,False,Excellent
6,Casa Azul Hostel,Manzanal 66 Corner Rosario Puerto Varas Chile,4.0,342,-36.969566,-72.086518,False,True,Very good
7,Enjoy Puerto Varas,Klenner 349 Puerto Varas Chile,4.0,970,-41.314247,-72.983703,False,True,Very good
8,Hotel Cabana del Lago,Klener 195 Pasaje Luis Wellmann Puerto Varas ...,4.5,950,-41.313501,-72.984756,True,False,Excellent
9,Solace Hotel,Imperial 0211 Puerto Varas Chile,4.5,821,-41.320709,-72.983904,True,False,Excellent


In [31]:
import geopandas as gpd
from shapely.geometry import Point

# Load the Shapefile
lago = gpd.read_file('./data/Masas_Lacustres/masas_lacustres.shp')
lago

,objectid,st_area_sh,st_length_,Nombre,Tipo,geometry
0,1,1.411784e+08,77845.633228,Salar de San Martín o Carcote,Salar,"POLYGON ((-7605868.923 -2425613.170, -7605850...."
1,2,3.087091e+08,147548.911324,Salar de Ascotan,Salar,"POLYGON ((-7597857.995 -2442164.143, -7597825...."
2,3,1.218202e+04,532.486240,None,Laguna,"POLYGON ((-7738411.433 -2494152.945, -7738388...."
3,4,1.439584e+06,12002.113642,Embalse Conchi,Embalse,"POLYGON ((-7637434.400 -2512504.257, -7637447...."
4,5,2.706124e+06,11821.430217,None,Salar,"POLYGON ((-7591081.555 -2511143.364, -7591057...."
...,...,...,...,...,...,...
16470,44898,1.453791e+04,503.982324,None,Tranque,"POLYGON ((-7851718.596 -3975116.573, -7851853...."
16471,44899,3.088426e+03,197.206234,None,Tranque,"POLYGON ((-7851990.035 -3974965.949, -7851985...."
16472,44900,3.007371e+03,194.601222,None,Tranque,"POLYGON ((-7851981.838 -3974895.953, -7851976...."
16473,44901,1.389800e+04,664.653430,None,Laguna,"POLYGON ((-7851166.692 -3975877.652, -7851166...."


In [42]:
import geopandas as gpd
from shapely.geometry import Point

# Load the Shapefile
lago = gpd.read_file('./data/Masas_Lacustres/masas_lacustres.shp')

# Coordenadas del hotel (reemplaza con las coordenadas reales)
# Coordenadas de iquique
solace_hotel_lejos = Point(-41.3207091,-72.98390389552812) # Solace Hotel
radison_cerca = Point(-41.3179833,-72.9817776) # Radison
park_radison_media = Point(-41.3149096,-72.9856539) # Park Radison

# Convert the points to GeoSeries
solace_hotel_lejos = gpd.GeoSeries([solace_hotel_lejos], crs='EPSG:4326')
radison_cerca = gpd.GeoSeries([radison_cerca], crs='EPSG:4326')
park_radison_media = gpd.GeoSeries([park_radison_media], crs='EPSG:4326')

# Convert the GeoSeries to the same CRS as the Shapefile
lago = lago.to_crs('EPSG:32719')  # reemplaza 'EPSG:4326' con el EPSG de tus coordenadas si es diferente
solace_hotel_lejos = solace_hotel_lejos.to_crs(lago.crs)
radison_cerca = radison_cerca.to_crs(lago.crs)
park_radison_media = park_radison_media.to_crs(lago.crs)

# Calcula la distancia mínima desde el hotel a cualquier punto del lago en metros
distancia_minima_solace = lago.distance(solace_hotel_lejos[0]).min()
distancia_minima_radison = lago.distance(radison_cerca[0]).min()
distancia_minima_park_radison = lago.distance(park_radison_media[0]).min()

print(f"La distancia mínima al lago más cercano desde el Solace Hotel es de {distancia_minima_solace:.2f} metros")
print(f"La distancia mínima al lago más cercano desde el Radison es de {distancia_minima_radison:.2f} metros")
print(f"La distancia mínima al lago más cercano desde el Park Radison es de {distancia_minima_park_radison:.2f} metros")


La distancia mínima al lago más cercano desde el Solace Hotel es de 2241992.91 metros
La distancia mínima al lago más cercano desde el Radison es de 2241892.69 metros
La distancia mínima al lago más cercano desde el Park Radison es de 2242267.27 metros


In [22]:
# Hacer regresion lineal con el df_procesado para saber que categorias influyen en el score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = df_procesado.drop(columns=['score','nombre','localidad','categorias'])
y = df_procesado['score']
# Hacer standarizacion
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Mostrar coeficientes y score pero con el nombre correspondiente a cada variable con el intercepto
coeficientes = pd.DataFrame(model.coef_, index=df_procesado.drop(columns=['score','nombre','localidad','categorias']).columns, columns=['Coeficiente'])
coeficientes.loc['Intercepto'] = model.intercept_
coeficientes
# Mostrar el score
y_pred = model.predict(X_test)
print('Score:', model.score(X_test, y_test))
print('MSE:', mean_squared_error(y_test, y_pred))



Score: 0.0
MSE: 0.0031790412939216076
